In [1]:
import numpy as np
import scipy as sp
import scipy.stats as stats
from scipy import linalg as LA
from math import sqrt

class IterativeMethod():
    def __init__(self, A, b):
        self.A = A
        self.b = b
        self.m, self.n = A.shape
        
        self.LU = self.A * (1 - np.eye(min(*A.shape)))
        self.U = self.LU.copy()
        for i in range(self.m):
            for j in range(i):
                self.U[i,j] = 0
        self.L = self.LU - self.U
        self.D = self.A - self.LU
        self.Dinv = LA.inv(self.D)
        
        self.BJ = - self.Dinv.dot(self.LU)
        self.bJ = self.Dinv.dot(self.b)

    def stop_criteria(self, x_prev, x_next, eps=1e-5):
        difference = LA.norm(x_prev - x_next) / (LA.norm(x_prev)+eps)
        accuracy = self.accuracy(x_next)
        if self.out:
            print('\trel. diff. = {:.4f}, acc. = {:.4f}'\
                  .format(difference, accuracy))
#         return difference < self.diff_eps
        return accuracy < self.acc_eps
    
    def accuracy(self, x):
        return LA.norm(self.A.dot(x) - self.b)
    
    def check_B(self):
        eigvalues, eigvectors = LA.eig(self.BJ)
        print(max(np.abs(eigvalues)))
    
    def run(self, x0, out=True, diff_eps=1e-5, acc_eps=1e-3, maxiter=1e3):
        self.out = out
        self.diff_eps = diff_eps
        self.acc_eps = acc_eps
        self.maxiter = maxiter
        if self.out:
            print('\tx_{} = {}'.format(0, x0.flatten()))
        x_prev = x0
        x_next = self.iteration(x_prev)
        it = 1
        while not self.stop_criteria(x_prev, x_next) and it < self.maxiter:
            it += 1
            x_prev = x_next
            x_next = self.iteration(x_prev)
            if out:
                print('\tx_{} = {}'.format(it-1, x_prev.flatten()))
        self.finalize(x_next, it)
        return self.x
    
    def finalize(self, x, it):
        self.it = it
        if self.it < self.maxiter:
            self.x = x
            self.acc = self.accuracy(x)
            print('The method converged in {} it. to x* = {}'\
                  .format(self.it, self.x.flatten()))
            print('The accuracy is {:.4f}'.format(self.acc))
        else:
            self.x = np.nan
            print('The method didn\'t converge.')

class Jacobi(IterativeMethod):

    def iteration(self, x_prev):
        x_next = self.BJ.dot(x_prev) + self.bJ
        return x_next


class GaussSeidel(IterativeMethod):    

    def iteration(self, x_prev):
        x_next = np.zeros_like(x_prev)
        Dinv = LA.inv(self.D)
        for i in range(x_next.shape[0]):
            x_next[i] = Dinv[i,i] * (self.b[i] - self.L[i,:].dot(x_next) 
                                               - self.U[i,:].dot(x_prev))
        return x_next

In [2]:
u = np.array([1,-1,1]).reshape((-1,1))
alpha = 2
A = alpha * np.eye(3) + u.dot(u.T)
b = u
x0 = np.zeros((3,1))

In [3]:
J = Jacobi(A, b)
x = J.run(x0, out=True, maxiter=1000, acc_eps=0.001)

	x_0 = [0. 0. 0.]
	rel. diff. = 57735.0269, acc. = 1.1547
	x_1 = [ 0.33333333 -0.33333333  0.33333333]
	rel. diff. = 0.6667, acc. = 0.7698
	x_2 = [ 0.11111111 -0.11111111  0.11111111]
	rel. diff. = 1.3333, acc. = 0.5132
	x_3 = [ 0.25925926 -0.25925926  0.25925926]
	rel. diff. = 0.3809, acc. = 0.3421
	x_4 = [ 0.16049383 -0.16049383  0.16049383]
	rel. diff. = 0.4102, acc. = 0.2281
	x_5 = [ 0.22633745 -0.22633745  0.22633745]
	rel. diff. = 0.1939, acc. = 0.1521
	x_6 = [ 0.1824417 -0.1824417  0.1824417]
	rel. diff. = 0.1604, acc. = 0.1014
	x_7 = [ 0.21170553 -0.21170553  0.21170553]
	rel. diff. = 0.0922, acc. = 0.0676
	x_8 = [ 0.19219631 -0.19219631  0.19219631]
	rel. diff. = 0.0677, acc. = 0.0451
	x_9 = [ 0.20520246 -0.20520246  0.20520246]
	rel. diff. = 0.0423, acc. = 0.0300
	x_10 = [ 0.19653169 -0.19653169  0.19653169]
	rel. diff. = 0.0294, acc. = 0.0200
	x_11 = [ 0.2023122 -0.2023122  0.2023122]
	rel. diff. = 0.0190, acc. = 0.0133
	x_12 = [ 0.19845853 -0.19845853  0.19845853]
	rel. dif

In [4]:
GS = GaussSeidel(A, b)
x = GS.run(x0, out=True, maxiter=1000, acc_eps=0.001)

	x_0 = [0. 0. 0.]
	rel. diff. = 42713.1948, acc. = 0.3989
	x_1 = [ 0.33333333 -0.22222222  0.14814815]
	rel. diff. = 0.3074, acc. = 0.0566
	x_2 = [ 0.20987654 -0.21399177  0.1920439 ]
	rel. diff. = 0.0496, acc. = 0.0082
	x_3 = [ 0.19798811 -0.20332266  0.19956307]
	rel. diff. = 0.0089, acc. = 0.0023
	x_4 = [ 0.19903809 -0.20046628  0.20016521]
	rel. diff. = 0.0025, acc. = 0.0006
The method converged in 5 it. to x* = [ 0.1997895  -0.2000151   0.20006513]
The accuracy is 0.0006


In [351]:
# eps = 10e-10
# alphas = np.linspace(-10,10,100)
# alphas = [-1/2,0,1,2,3]
# b = np.array([1,-1,1]).reshape((-1,1))
# for a in alphas:
#     L = np.array([[0, 0, 0],
#                   [-1, 0, 0],
#                   [1, -1, 0]])
#     D = np.array([[1+a, 0, 0],
#                   [0, 1+a, 0],
#                   [0, 0, 1+a]])
#     U = np.array([[0,-1,1],
#                   [0,0,-1],
#                   [0,0,0]])
#     LD = L+D
#     LDinv = np.array([[(1+a)**2, 0, 0],
#                   [1+a, (1+a)**2, 0],
#                   [-a, 1+a, (1+a)**2]]) / (1+a)**3
#     assert ((LD.dot(LDinv)-np.eye(3))<eps).all() and ((LDinv.dot(LD)-np.eye(3))<eps).all()
#     Dinv = np.array([[1, 0, 0],
#                   [0, 1, 0],
#                   [0, 0, 1]]) / (1+a)

#     B_J = - Dinv.dot(L+U)
#     B_J_ = np.array([[0,1,-1],
#                       [1,0,1],
#                       [-1,1,0]]) / (1+a)
# #     print(B_J-B_J_)
#     b_J = Dinv.dot(b)
#     b_J_ = np.array([[1,-1,1]]).T / (1+a)
# #     print(b_J-b_J_)
    
#     B_GS = - LDinv.dot(U)
#     B_GS_ = np.array([[0,(1+a)**2,-(1+a)**2],
#                       [0,1+a,a*(1+a)],
#                       [0,-a,1+2*a]]) / (1+a)**3
# #     print(B_GS-B_GS_)
#     b_GS = LDinv.dot(b)
#     b_GS_ = np.array([[(1+a)**2, -a*(1+a), a**2]]).T / (1+a)**3
# #     print(b_GS-b_GS_)
    
#     print(LA.eig(B_GS)[0])
#     print(np.roots([1,-(2+3*alpha)/(1+alpha)**3,1/(1+alpha)**3]))